In [1]:
import sys
print(sys.executable)
import strax
import straxen
print('Straxen: ', straxen.__version__, straxen.__file__)
print('Strax: ', strax.__version__, strax.__file__)
import configparser as cp

sys.path.append('/home/gvolta/XENONnT/gain_spe')
from PMTanalysis import *
PMT = PMTanalyzer('/home/gvolta/XENONnT/gain_spe')

/dali/lgrandi/strax/miniconda3/envs/strax/bin/python
Straxen:  0.7.3 /home/gvolta/XENONnT/straxen/straxen/__init__.py
Strax:  0.10.3 /dali/lgrandi/strax/strax/strax/__init__.py




___   ___  _______ .__   __.   ______   .__   __.        .___________.   __  __             _____    _____    ____  
\  \ /  / |   ____||  \ |  |  /  __  \  |  \ |  |        |           |  |  \/  |    /\     / ____|  |_   _| /  ___| 
 \  V  /  |  |__   |   \|  | |  |  |  | |   \|  |  _   _ `---|  |----`  | \  / |   /  \    | |  __    | |   | |      
  >   <   |   __|  |  . `  | |  |  |  | |  . `  | | \ | |    |  |       | |\/| |  / /\ \   | | |_ |   | |   | |      
 /  .  \  |  |____ |  |\   | |  `--`  | |  |\   | |  \| |    |  |       | |  | | / ____ \  | |__| |  _| |_  | |___  
/__/ \__\ |_______||__| \__|  \______/  |__| \__| |_|\__|    |__|       |_|  |_|/_/    \_\ \ _____| |_____| \_____|  
powered by Giovanni Volta and Chiara Capelli (c)




Configuration file:  /home/gvolta/XENONnT/gain_spe/configurat

In [2]:
strax.Mailbox.DEFAULT_MAX_MESSAGES = 2
st = straxen.contexts.xenonnt_online()

In [3]:
run = st.select_runs(run_id='7212')
run

Checking data availability: 100%|██████████| 1/1 [00:13<00:00, 13.04s/it]


,number,mode,start,end,name,tags,livetime,trigger.events_built,strax_defaults,reader.ini.name,tags.name,raw_records_available
7212,7212,,2020-04-15 15:38:19.055,2020-04-15 15:38:49.345,7212,,00:00:30.290000,NaN,NaN,NaN,NaN,False


In [4]:
!ls /dali/lgrandi/xenonnt/raw/

007154-raw_records_aqmon-rfzvpzj4mf  007193-raw_records_mv-rfzvpzj4mf
007154-raw_records-rfzvpzj4mf	     007193-raw_records-rfzvpzj4mf
007157-raw_records-rfzvpzj4mf	     007194-raw_records_aqmon-rfzvpzj4mf
007158-raw_records_aqmon-rfzvpzj4mf  007194-raw_records_he-rfzvpzj4mf
007158-raw_records_he-rfzvpzj4mf     007194-raw_records_mv-rfzvpzj4mf
007158-raw_records-rfzvpzj4mf	     007194-raw_records-rfzvpzj4mf
007159-raw_records_aqmon-rfzvpzj4mf  007196-raw_records_aqmon-rfzvpzj4mf
007159-raw_records_he-rfzvpzj4mf     007196-raw_records_he-rfzvpzj4mf
007159-raw_records-rfzvpzj4mf	     007196-raw_records_mv-rfzvpzj4mf
007177-raw_records_aqmon-rfzvpzj4mf  007196-raw_records-rfzvpzj4mf
007177-raw_records_he-rfzvpzj4mf     007197-raw_records_aqmon-rfzvpzj4mf
007177-raw_records_mv-rfzvpzj4mf     007197-raw_records_he-rfzvpzj4mf
007177-raw_records-rfzvpzj4mf	     007197-raw_records_mv-rfzvpzj4mf
007178-raw_records_aqmon-rfzvpzj4mf  007197-raw_records-rfzvpzj4mf
007178-raw_records_he-rfzvpzj4mf 

In [ ]:
data_rr = st.get_array('007212', 'raw_records', second_range=(0,2))

/dali/lgrandi/strax/strax/strax/context.py:205: UserWarning: Unknown config option second_range; will do nothing.
  warnings.warn(f"Unknown config option {k}; will do nothing.")
/dali/lgrandi/strax/strax/strax/context.py:211: UserWarning: Invalid context option second_range; will do nothing.
  warnings.warn(f"Invalid context option {k}; will do nothing.")
